<a href="https://colab.research.google.com/github/Mardoc21/Central-SSA/blob/main/hacker_quem_fe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# =========================
# VGP COLAB - DRIVE INBOX WHATSAPP (PRODUÇÃO - V2.1)
# MELHORIAS:
#  (1) Usa GPU T4 quando disponível (torch.cuda)
#  (2) Remove 403 "fileNotDownloadable" pulando Google Docs/Sheets/etc
#  (3) Limpa cache local (apaga ZIP local após extrair)
#  (4) Terminal enxuto + PROGRESS BAR (tqdm) + heartbeat
#  (5) Ao terminar: mostra caminhos, lista arquivos, imprime run_log e prévia do index.csv
# =========================

from google.colab import drive
drive.mount('/content/drive')

!pip -q install faster-whisper google-api-python-client google-auth-httplib2 google-auth-oauthlib torch tqdm
!apt-get -qq update && apt-get -qq install -y ffmpeg

import os, io, re, json, csv, zipfile, hashlib, traceback, time
from pathlib import Path
from datetime import datetime

from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

import torch
from faster_whisper import WhisperModel
from tqdm.auto import tqdm

# =========================
# CONFIG (AJUSTE AQUI)
# =========================
FOLDER_ID = "1hQn6Vp2LG_qFaLQLMiL60Aq_ifTHga-N"     # ID da pasta Inbox no Drive
OUT_BASE  = Path("/content/drive/MyDrive/VGP_OUT")   # Saída no Drive
DL_DIR    = Path("/content/vgp_download")            # Cache local (Colab)

MODEL_SIZE = "small"   # tiny / base / small / medium / large-v3
LANG = "pt"
VAD_FILTER = True

MOVE_ZIPS_TO_PROCESSED = False
DELETE_LOCAL_ZIPS_AFTER_EXTRACT = True

# Progress/prints
PRINT_EVERY = 50              # imprime a cada 50 transcrições (além da barra)
HEARTBEAT_SECONDS = 60        # a cada 60s imprime “ainda rodando” (útil p/ long run)

AUDIO_EXTS = {".opus",".ogg",".mp3",".m4a",".wav",".mp4",".webm",".mkv",".mov",".aac",".flac"}

GOOGLE_APP_MIMES = {
  "application/vnd.google-apps.document",
  "application/vnd.google-apps.spreadsheet",
  "application/vnd.google-apps.presentation",
  "application/vnd.google-apps.folder",
  "application/vnd.google-apps.shortcut",
  "application/vnd.google-apps.form",
  "application/vnd.google-apps.drawing",
  "application/vnd.google-apps.site",
}

ZIP_MIMES = {
    "application/zip",
    "application/x-zip-compressed",
    "application/octet-stream",
}

# =========================
# PATHS
# =========================
OUT_BASE.mkdir(parents=True, exist_ok=True)
DL_DIR.mkdir(parents=True, exist_ok=True)

TRANSCRIPTS_DIR = OUT_BASE / "transcripts"
EXTRACT_DIR     = OUT_BASE / "whatsapp_extracted"
LOGS_DIR        = OUT_BASE / "logs"

TRANSCRIPTS_DIR.mkdir(parents=True, exist_ok=True)
EXTRACT_DIR.mkdir(parents=True, exist_ok=True)
LOGS_DIR.mkdir(parents=True, exist_ok=True)

HASH_DB_PATH    = TRANSCRIPTS_DIR / "processed_hashes.json"
INDEX_CSV_PATH  = TRANSCRIPTS_DIR / "index.csv"
RUN_LOG_PATH    = LOGS_DIR / f"run_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

# =========================
# UTILS
# =========================
def now():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def sha256_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        for chunk in iter(lambda: f.read(1024 * 1024), b""):
            h.update(chunk)
    return h.hexdigest()

def safe_name(name: str) -> str:
    name = name.replace("/", "_").replace("\\", "_")
    name = re.sub(r"[\x00-\x1f]+", "_", name)
    name = name.strip()
    return name[:180] if len(name) > 180 else name

def is_zip_by_signature(path: Path) -> bool:
    try:
        with path.open("rb") as f:
            sig = f.read(2)
        return sig == b"PK"
    except:
        return False

def extract_zip(zip_path: Path, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(out_dir)

def load_hash_db() -> set:
    if HASH_DB_PATH.exists():
        try:
            return set(json.loads(HASH_DB_PATH.read_text(encoding="utf-8")))
        except:
            return set()
    return set()

def save_hash_db(s: set):
    HASH_DB_PATH.write_text(json.dumps(sorted(list(s))), encoding="utf-8")

def append_index_rows(rows):
    existing = {}
    if INDEX_CSV_PATH.exists():
        with INDEX_CSV_PATH.open("r", encoding="utf-8") as f:
            r = csv.DictReader(f)
            for row in r:
                existing[row["hash"]] = row

    for row in rows:
        existing[row["hash"]] = row

    with INDEX_CSV_PATH.open("w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["hash","file","source","text","ts"])
        w.writeheader()
        for k in existing:
            w.writerow(existing[k])

def tail_lines(path: Path, n=30):
    try:
        lines = path.read_text(encoding="utf-8", errors="ignore").splitlines()
        return "\n".join(lines[-n:])
    except:
        return ""

def preview_index_csv(path: Path, n=10):
    if not path.exists():
        return []
    out = []
    with path.open("r", encoding="utf-8") as f:
        r = csv.DictReader(f)
        for i, row in enumerate(r):
            out.append(row)
            if i+1 >= n:
                break
    return out

# =========================
# GOOGLE DRIVE API
# =========================
auth.authenticate_user()
creds, project = default()
drive_svc = build("drive", "v3", credentials=creds)

def download_file(file_id: str, dest: Path):
    request = drive_svc.files().get_media(fileId=file_id)
    fh = io.FileIO(dest, "wb")
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()

def list_all_files_recursive(folder_id: str):
    files = []
    stack = [folder_id]
    while stack:
        fid = stack.pop()
        page_token = None
        while True:
            res = drive_svc.files().list(
                q=f"'{fid}' in parents and trashed=false",
                fields="nextPageToken, files(id,name,mimeType,parents)",
                pageToken=page_token
            ).execute()
            for f in res.get("files", []):
                if f["mimeType"] == "application/vnd.google-apps.folder":
                    stack.append(f["id"])
                else:
                    files.append(f)
            page_token = res.get("nextPageToken")
            if not page_token:
                break
    return files

def ensure_processed_folder(parent_folder_id: str) -> str:
    q = f"'{parent_folder_id}' in parents and trashed=false and mimeType='application/vnd.google-apps.folder' and name='_PROCESSED'"
    res = drive_svc.files().list(q=q, fields="files(id,name)").execute()
    fs = res.get("files", [])
    if fs:
        return fs[0]["id"]
    meta = {"name": "_PROCESSED", "mimeType": "application/vnd.google-apps.folder", "parents": [parent_folder_id]}
    created = drive_svc.files().create(body=meta, fields="id").execute()
    return created["id"]

def move_file_to_folder(file_id: str, new_parent_id: str):
    file = drive_svc.files().get(fileId=file_id, fields="parents").execute()
    prev_parents = ",".join(file.get("parents", []))
    drive_svc.files().update(
        fileId=file_id,
        addParents=new_parent_id,
        removeParents=prev_parents,
        fields="id, parents"
    ).execute()

# =========================
# WHISPER (GPU T4 quando disponível)
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"
print("device:", device, "| compute_type:", compute_type)

model = WhisperModel(MODEL_SIZE, device=device, compute_type=compute_type)

def transcribe_audio(file_path: Path) -> dict:
    segments, info = model.transcribe(
        str(file_path),
        language=LANG,
        vad_filter=VAD_FILTER
    )
    text = " ".join([s.text.strip() for s in segments]).strip()
    return {
        "file": file_path.name,
        "text": text,
        "language": getattr(info, "language", LANG),
        "ts": now()
    }

# =========================
# RUN PIPELINE
# =========================
run_log = {
    "ts_start": now(),
    "folder_id": FOLDER_ID,
    "model": MODEL_SIZE,
    "device": device,
    "compute_type": compute_type,
    "downloaded_files": 0,
    "skipped_google_files": 0,
    "zip_candidates": 0,
    "extracted_audio": 0,
    "loose_audio": 0,
    "to_transcribe": 0,
    "transcribed_new": 0,
    "skipped_already_done": 0,
    "errors": []
}

t0 = time.time()
last_heartbeat = time.time()

print("Listando arquivos do folder (recursivo)...")
files = list_all_files_recursive(FOLDER_ID)
print("Total arquivos encontrados:", len(files))

processed_hashes = load_hash_db()

downloaded = []       # [(local_path, meta)]
zip_candidates = []   # [{"path":Path, "drive_id":str, "name":str}]

# 1) DOWNLOAD (com barra)
pbar_dl = tqdm(files, desc="Baixando binários do Drive", unit="arquivo")
for f in pbar_dl:
    try:
        name = safe_name(f["name"])
        fid  = f["id"]
        mime = f.get("mimeType", "")

        if mime in GOOGLE_APP_MIMES:
            run_log["skipped_google_files"] += 1
            continue

        dest = DL_DIR / name
        if dest.exists():
            dest = DL_DIR / (f"{dest.stem}_{fid[:8]}{dest.suffix}")

        download_file(fid, dest)
        downloaded.append((dest, f))
        run_log["downloaded_files"] += 1

        if (mime in ZIP_MIMES) or (dest.suffix.lower() == ".zip") or is_zip_by_signature(dest):
            zip_candidates.append({"path": dest, "drive_id": fid, "name": name})

    except Exception as e:
        run_log["errors"].append({"stage": "download", "file": f.get("name","?"), "err": str(e)})
        continue

run_log["zip_candidates"] = len(zip_candidates)
print("\nResumo download:")
print("  Baixados:", run_log["downloaded_files"])
print("  Pulados (Google Docs/Sheets/etc):", run_log["skipped_google_files"])
print("  ZIPs detectados (robusto):", run_log["zip_candidates"])

# 2) EXTRAIR ZIPs (com barra)
extracted_audio = []
extracted_txt   = []

pbar_zip = tqdm(zip_candidates, desc="Extraindo ZIPs", unit="zip")
for z in pbar_zip:
    z_path = z["path"]
    z_out = EXTRACT_DIR / z_path.stem
    try:
        extract_zip(z_path, z_out)
        for p in z_out.rglob("*"):
            if p.is_file():
                ext = p.suffix.lower()
                if ext in AUDIO_EXTS:
                    extracted_audio.append(p)
                elif ext == ".txt":
                    extracted_txt.append(p)

        if DELETE_LOCAL_ZIPS_AFTER_EXTRACT:
            try:
                z_path.unlink()  # limpa cache local do Colab
            except:
                pass

    except Exception as e:
        run_log["errors"].append({"stage": "extract_zip", "zip": z_path.name, "err": str(e)})

# 3) Áudios soltos (fora dos ZIPs)
loose_audio = []
for dest, meta in downloaded:
    if dest.suffix.lower() in AUDIO_EXTS:
        loose_audio.append(dest)

run_log["extracted_audio"] = len(extracted_audio)
run_log["loose_audio"] = len(loose_audio)

all_audio = extracted_audio + loose_audio

# 4) Dedup por hash (com barra)
audio_unique = []
seen_audio_hash = set()

pbar_hash = tqdm(all_audio, desc="Hash/Dedup (SHA256)", unit="midia")
for p in pbar_hash:
    try:
        h = sha256_file(p)
        if h in seen_audio_hash:
            continue
        seen_audio_hash.add(h)
        audio_unique.append((p, h))
    except Exception as e:
        run_log["errors"].append({"stage": "hash_audio", "file": str(p), "err": str(e)})

run_log["to_transcribe"] = len(audio_unique)

print("\nResumo extração/dedup:")
print("  Áudios extraídos:", len(extracted_audio))
print("  Áudios soltos:", len(loose_audio))
print("  Áudios únicos p/ transcrever:", run_log["to_transcribe"])
print("  TXTs extraídos:", len(extracted_txt))

# 5) Transcrever incremental (com barra + heartbeat)
index_rows = []

pbar_tr = tqdm(audio_unique, desc="Transcrevendo (incremental)", unit="audio")
for i, (p, h) in enumerate(pbar_tr, 1):
    try:
        out_json = TRANSCRIPTS_DIR / f"{h}.json"

        if h in processed_hashes and out_json.exists():
            run_log["skipped_already_done"] += 1
            data = json.loads(out_json.read_text(encoding="utf-8"))
        else:
            # prints enxutos (além da barra)
            if (i == 1) or (i % PRINT_EVERY == 0):
                print(f"[{i}/{len(audio_unique)}] transcrevendo: {p.name}")

            data = transcribe_audio(p)
            data["hash"] = h
            data["source_path"] = str(p)
            out_json.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")
            processed_hashes.add(h)
            run_log["transcribed_new"] += 1

        index_rows.append({
            "hash": h,
            "file": data.get("file", p.name),
            "source": data.get("source_path", str(p)),
            "text": data.get("text", ""),
            "ts": data.get("ts", now())
        })

        # HEARTBEAT: garante que você vê que está vivo mesmo sem prints
        if time.time() - last_heartbeat >= HEARTBEAT_SECONDS:
            elapsed = int(time.time() - t0)
            print(f"…rodando ✅ | elapsed={elapsed}s | novos={run_log['transcribed_new']} | pulados={run_log['skipped_already_done']}")
            last_heartbeat = time.time()

    except Exception as e:
        run_log["errors"].append({"stage": "transcribe", "file": str(p), "err": str(e), "trace": traceback.format_exc()})

# 6) Persistir hash db + index consolidado
save_hash_db(processed_hashes)
append_index_rows(index_rows)

# 7) (Opcional) mover ZIPs detectados para _PROCESSED no Drive
if MOVE_ZIPS_TO_PROCESSED and zip_candidates:
    try:
        processed_folder_id = ensure_processed_folder(FOLDER_ID)
        moved = 0
        for z in zip_candidates:
            try:
                move_file_to_folder(z["drive_id"], processed_folder_id)
                moved += 1
            except Exception as e:
                run_log["errors"].append({"stage": "move_zip", "zip": z["name"], "err": str(e)})
        run_log["moved_zips_to_processed"] = moved
    except Exception as e:
        run_log["errors"].append({"stage": "ensure_processed_folder", "err": str(e)})

run_log["ts_end"] = now()
RUN_LOG_PATH.write_text(json.dumps(run_log, ensure_ascii=False, indent=2), encoding="utf-8")

# 8) Resumo final curto + “abre/mostra” a pasta (na prática: lista e prévias)
summary = {
  "start": run_log["ts_start"],
  "end": run_log["ts_end"],
  "downloaded": run_log["downloaded_files"],
  "skipped_google": run_log["skipped_google_files"],
  "zip": run_log["zip_candidates"],
  "unique_audio": run_log["to_transcribe"],
  "new": run_log["transcribed_new"],
  "skipped": run_log["skipped_already_done"],
  "errors": len(run_log["errors"]),
  "device": run_log["device"],
  "compute": run_log["compute_type"],
}

print("\nOK ✅ FINALIZADO")
print(summary)

print("\n=== PASTAS (Drive) ===")
print("Transcripts :", TRANSCRIPTS_DIR)
print("Index CSV   :", INDEX_CSV_PATH)
print("Hash DB     :", HASH_DB_PATH)
print("Extraídos   :", EXTRACT_DIR)
print("Log         :", RUN_LOG_PATH)

print("\n=== LISTAGEM (últimos 20) ===")
try:
    tr_files = sorted([p.name for p in TRANSCRIPTS_DIR.glob("*.json")])[-20:]
    print("transcripts/*.json:", tr_files)
except:
    pass

print("\n=== PREVIEW index.csv (10 linhas) ===")
try:
    for row in preview_index_csv(INDEX_CSV_PATH, n=10):
        # mostra curto
        txt = (row.get("text","") or "")[:120].replace("\n"," ")
        print(f"- {row.get('file','?')} | {row.get('ts','?')} | {txt}...")
except Exception as e:
    print("Falha preview index.csv:", e)

print("\n=== RUN_LOG (resumo) ===")
try:
    print(json.dumps(run_log, ensure_ascii=False, indent=2)[:2000])  # corta pra não poluir
    if len(json.dumps(run_log, ensure_ascii=False)) > 2000:
        print("... (run_log truncado na tela; arquivo completo salvo no caminho acima)")
except:
    pass

print("\n=== ABRIR PASTA NO COLAB (FILE BROWSER) ===")
# Isso não "abre no Drive" automático, mas abre o navegador de arquivos do Colab apontando a pasta:
!ls -lah "{OUT_BASE}"
!ls -lah "{TRANSCRIPTS_DIR}" | tail -n 30


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
device: cuda | compute_type: float16


Listando arquivos do folder (recursivo)...
Total arquivos encontrados: 687


Baixando binários do Drive:   0%|          | 0/687 [00:00<?, ?arquivo/s]


Resumo download:
  Baixados: 684
  Pulados (Google Docs/Sheets/etc): 3
  ZIPs detectados (robusto): 29


Extraindo ZIPs:   0%|          | 0/29 [00:00<?, ?zip/s]

Hash/Dedup (SHA256):   0%|          | 0/8474 [00:00<?, ?midia/s]


Resumo extração/dedup:
  Áudios extraídos: 7932
  Áudios soltos: 542
  Áudios únicos p/ transcrever: 4821
  TXTs extraídos: 41


Transcrevendo (incremental):   0%|          | 0/4821 [00:00<?, ?audio/s]

[1/4821] transcrevendo: PTT-20240718-WA0015.opus
…rodando ✅ | elapsed=5614s | novos=1 | pulados=0
…rodando ✅ | elapsed=5674s | novos=32 | pulados=0
[50/4821] transcrevendo: PTT-20240813-WA0009.opus
…rodando ✅ | elapsed=5735s | novos=65 | pulados=0
[100/4821] transcrevendo: PTT-20241015-WA0059.opus
…rodando ✅ | elapsed=5795s | novos=100 | pulados=0
…rodando ✅ | elapsed=5856s | novos=146 | pulados=0
[150/4821] transcrevendo: PTT-20241024-WA0071.opus
…rodando ✅ | elapsed=5916s | novos=195 | pulados=0
[200/4821] transcrevendo: PTT-20250408-WA0007.opus
…rodando ✅ | elapsed=5976s | novos=246 | pulados=0
[250/4821] transcrevendo: VID-20251231-WA0017.mp4
[300/4821] transcrevendo: PTT-20251125-WA0032.opus
…rodando ✅ | elapsed=6036s | novos=328 | pulados=0
[350/4821] transcrevendo: PTT-20260202-WA0004.opus
…rodando ✅ | elapsed=6099s | novos=390 | pulados=0
[400/4821] transcrevendo: PTT-20231011-WA0062.opus
…rodando ✅ | elapsed=6162s | novos=429 | pulados=0
[450/4821] transcrevendo: PTT-20240726-